<a href="https://colab.research.google.com/github/Viktory4121/MORD/blob/main/%D0%94%D0%9E%D0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Динамические окрестностные модели (ДОМ)
####Вариант 5

In [7]:
import numpy as np
import pandas as pd
import random
import copy
import math
from prettytable import PrettyTable
import itertools as it

In [ ]:
#A={a1, a2, a3}

#какие дуги входят в позицию:
#Ox[a1]={a1}
#Ox[a2]={}
#Ox[a3]={a1, a2}

#в какие позиции входят v
#Ov[a1]={a1}
#Ov[a2]={a2}
#Ov[a3]={a3}

In [8]:
#количества состояний/входов/выходов
count = [3, 3, 3]
Sx = np.array([[1, 0, 1], [0, 0, 1], [0, 0, 0]])
Sv = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

print("Матрица смежности:")
print("Sx = ")
print(Sx)
print("Sv = ")
print(Sv)

Матрица смежности:
Sx = 
[[1 0 1]
 [0 0 1]
 [0 0 0]]
Sv = 
[[1 0 0]
 [0 1 0]
 [0 0 1]]


In [23]:
def x1(x, v):
  return math.sin(x[0]**2 + math.log(abs(v[1] + v[0])) / x[2])

def x2(x, v):
  return math.cos(v[0] * x[2] - (x[1] / x[2])**3)

def x3(x, v):
  return math.sin(math.tan(x[2] * v[2]) - 3**(v[0]))

def v1(x, v):
  return (x[1] * v[0] * v[1] + math.tan(x[2])**2)

def v2(x, v):
  return (0.2 * v[0] + 33.0 * v[1] + v[2])

def v3(x, v):
  return (math.exp(x[0]) + .45 * math.log2(abs(x[1] * x[2])))

def y1(x, v):
  return x[0] * v[0] * math.sqrt(abs(x[0] / x[1]))

def y2(x, v):
  return math.sqrt(abs(math.sin(v[2])))

def y3(x, v):
  return v[2] * v[0] + 4

In [24]:
#Метод псевдообращения Бена-Израиля
def BenIsrael(A, eps):
  I = np.diag(np.diag(np.ones([len(A),len(A)])))
  buf = 1
  i = 1

  if(np.linalg.norm(A, ord='fro') != 0): #если не нулевая норма
    gamma = 1.6 / (np.linalg.norm(A, ord='fro')**2)
    A1 = gamma * (A.T)
    i += 1
    A2 = A1.dot(2*I - A.dot(A1))
    E = np.linalg.norm(A2 - A1, ord=np.inf) / np.linalg.norm(A1, ord=np.inf)
  else: #если нулевая норма
    A1 = np.zeros((len(A[0]), len(A)))
    buf = 0

  while(E >= eps and buf):
    A1 = A2.copy()
    A2 = A1.dot(2*I - A.dot(A1))
    E = np.linalg.norm(A2 - A1, ord=np.inf) / np.linalg.norm(A1, ord=np.inf)
    i += 1

  return A1

#=================================================================
#Генерация случайного вектора из 3-х элементов в диапазоне от -3 до 3
def generationRandomVector(elem = 3, start = -3, end = 3):
  vector = []
  for i in range(elem):
    vector.append(random.uniform(start, end))
  return vector

#=================================================================
#создание столбца таблицы
def createVariants(function, x, v, typeFunc = 1, elem = 100):
  vector = []

  if(typeFunc == 0):
    for i in range(elem):
      vector.append(function(generationRandomVector(), generationRandomVector()))
  else:
    for i in range(elem):
      vector.append(function(x[i], v[i]))

  return vector

#=================================================================
#count(x, v, y)
def createTableDOM(functions, count, step, oldTable = [], elem = 100):
  table = []
  vector = []
  columns = createColumns(step, count)
  if(step == 0):
    for i in range(sum(count[0:2])):
      table.append(createVariants(functions[i], [], [], 0)) #xi, vi

  else:
    for i in range(count[0]): #xi
      table.append(createVariants(functions[i], (np.matrix(oldTable).T)[0:count[0]], (np.matrix(oldTable).T)[count[0]:sum(count[0:2])]))

    for i in range(count[1]): #vi
      table.append(createVariants(functions[i], [], [], 0))


  for i in range(count[2]): #yi
    table.append(createVariants(functions[i], (np.matrix(table).T)[0:count[0]], (np.matrix(table).T)[count[0]:sum(count[0:2])]))



  table = pd.DataFrame(np.matrix(table).T, columns = columns)
  return table

def createColumns(step, count):
  columns = []
  for i in range(count[0]):
    columns.append("x" + str(i + 1) + "(" + str(step) + ")")
  for i in range(count[1]):
    columns.append("v" + str(i + 1) + "(" + str(step) + ")")
  for i in range(count[2]):
    columns.append("y" + str(i + 1) + "(" + str(step) + ")")
  return columns

In [25]:
#functions = [x1, x2, x3, v1, v2, v3, y1, y2, y3]
functions = [x1, x2, x3, v1, v2, v3, y1, y2, y3]
createTableDOM(functions, count, 0)

LinAlgError: ignored